<a href="https://colab.research.google.com/github/jatinparab/langchain-examples/blob/main/math_search_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install langchain huggingface_hub openai google-search-results tiktoken wikipedia

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI

In [ ]:
llm = OpenAI(temperature=0)

In [ ]:
tools = load_tools(["serpapi", "llm-math"], llm=llm)

In [ ]:
agent = initialize_agent(tools,
                         llm,
                         agent="zero-shot-react-description",
                         verbose=True)

In [ ]:
agent.run("Hi How are you today?")

In [ ]:
agent.run("Who is the United States President? What is his current age raised divided by 2?")


In [ ]:
agent.run("What is the average age in the United States? How much less is that then the age of the current US President?")
